# WGBS DMR analysis

This notebook provides codes for WGBS DMR analysis.

1. 

In [1]:
import pandas as pd
import seaborn as sns
%matplotlib inline

import glob
import os
import pickle
import random
from collections import Counter, defaultdict

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import pybedtools as pb
import seaborn as sns
import tqdm
from scipy import histogram2d
from scipy.special import beta as B
from scipy.special import comb as C
from scipy.special import digamma
from scipy.special import gammaln as logG
from scipy.special import polygamma
from sklearn.mixture import GaussianMixture

%matplotlib inline

import cycler

scale = 1.3
plt.rc('axes', linewidth=1.33, labelsize=14)
plt.rc('xtick', labelsize=10 * scale)
plt.rc('ytick', labelsize=10 * scale)

plt.rc('xtick', bottom=True)
plt.rc('xtick.major', size=5 * scale, width=1.33)
plt.rc('xtick.minor', size=5 * scale, width=1.33)

plt.rc('ytick', left=True)
plt.rc('ytick.major', size=5 * scale, width=1.33)
plt.rc('ytick.minor', size=5 * scale, width=1.33)

plt.rc('legend', fontsize=7 * scale)
plt.rc('grid', color='grey', linewidth=0.5, alpha=0.33)
plt.rc('font', family='FreeSans')

color_palette = [
    '#005AC8',
    '#AA0A3C',
    '#0AB45A',
    '#FA7850',
    '#8214A0',
    '#FA78FA',
    '#A0FA82',
    '#006E82',
    '#00A0FA',
    '#14D2DC',
    '#F0F032',
    '#FAE6BE',
]

mpl.rcParams['axes.prop_cycle'] = cycler.cycler(color=color_palette)

## 1. Compute proportions of DMRs in whole genome/promoters/enhancers (Fig. 1j)

Here we show how we computed the proportions of hypo-/hyper- DMRs with respect to the whole genome/promoters/enhancers.

### Load DMR intervals

In [19]:
hypo_dmrs = pd.read_csv('data/ad_hypo.dmr.final.bed', sep='\t', names=['chrom', 'start', 'end']) # 5451 entries
hyper_dmrs = pd.read_csv('data/ad_hyper.dmr.final.bed', sep='\t', names=['chrom', 'start', 'end']) # 934 entries

promoter_hypo_dmrs = pd.read_csv('data/transcript_promoterup1000_overlapping_hypo_dmr.bed', sep='\t', names=['chrom', 'start', 'end']) # 589 entries
promoter_hyper_dmrs = pd.read_csv('data/transcript_promoterup1000_overlapping_hyper_dmr.bed', sep='\t', names=['chrom', 'start', 'end']) # 82 entries

enhancer_hypo_dmrs = pd.read_csv('data/enhancer_overlapping_hypo_dmr.bed', sep='\t', names=['chrom', 'start', 'end']) # 1393 entries
enhancer_hyper_dmrs = pd.read_csv('data/enhancer_overlapping_hyper_dmr.bed', sep='\t', names=['chrom', 'start', 'end']) # 155 entries

hypo_dmrs.shape, hyper_dmrs.shape, promoter_hypo_dmrs.shape, promoter_hyper_dmrs.shape,\
enhancer_hypo_dmrs.shape, enhancer_hyper_dmrs.shape

((5451, 3), (934, 3), (589, 3), (82, 3), (1393, 3), (155, 3))

### Load promoters and enhancers

In [20]:
promoters = pd.read_csv('data/promoters.mm10.transcript.bed', sep='\t', names=['chrom', 'start', 'end', 'gene'])
promoters = promoters[promoters.chrom != 'chrM']

hypo_dmr_overlapping_promoters = pd.read_csv('data/hypo_dmr_overlapping_transcript_promoter_up1000.bed', sep='\t', names=['chrom', 'start', 'end', 'gene']) # 1377 entries
hyper_dmr_overlapping_promoters = pd.read_csv('data/hyper_dmr_overlapping_transcript_promoter_up1000.bed', sep='\t', names=['chrom', 'start', 'end', 'gene']) # 150 entries

len(set(promoters.gene)), hypo_dmr_overlapping_promoters.shape, hyper_dmr_overlapping_promoters.shape

(54715, (1377, 4), (150, 4))

In [25]:
enhancers = pd.read_csv('data/enhancers.bed', sep='\t', names=['chrom', 'start', 'end', 'name'])

In [90]:
enhancers = pd.concat([
    pd.read_csv('result/bed/cebp_enhancer.bed', sep='\t', names=['chrom', 'start', 'end', 'name', 'score', 'strand']),
    pd.read_csv('result/bed/non_cebp_enhancer.bed', sep='\t', names=['chrom', 'start', 'end', 'name', 'score', 'strand']),
])

In [91]:
len(enhancers)

137995

In [92]:
enhancers.sort_values(['chrom', 'start', 'end'])[['chrom', 'start', 'end', 'name']].to_csv('result/bed/enhancers.bed', sep='\t', index=False, header=False)

In [99]:
enhancers.chrom.unique()

array(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr3', 'chr4', 'chr5',
       'chr6', 'chr7', 'chr8', 'chr9', 'chrX'], dtype=object)

### Compute the size of whole mm10 genome

We simply read mm10 fasta file and counted the number of whole bases.

As the size of `mm10.fa` is too large for GitHub, we just assign the precomputed value to `n_bases_total` and commented out the procedures of computing it.

In [27]:
# from Bio import SeqIO
# mm10 = SeqIO.to_dict(SeqIO.parse('data/mm10.fa', 'fasta'))
# chromosomes = [f'chr{i}' for i in range(1, 20)] + ['chrX', 'chrY']
# n_bases_total = 0
# for chrom in chromosomes:
#     n_bases_total += len(mm10[chrom].seq)

n_bases_total = 2725521370

### Compute the proportion of hypo-/hyper-DMRs with respect to the whole genome

In [34]:
n_bases_hypo_dmr = (hypo_dmrs.end - hypo_dmrs.start).sum()
n_bases_hyper_dmr = (hyper_dmrs.end - hyper_dmrs.start).sum()

print(f'% Hypo-DMR w.r.t Whole genome = {n_bases_hypo_dmr / n_bases_total:.2%}')
print(f'% Hyper-DMR w.r.t Whole genome = {n_bases_hyper_dmr / n_bases_total:.2%}')

% Hypo-DMR w.r.t Whole genome = 0.48%
% Hyper-DMR w.r.t Whole genome = 0.04%


### Compute the total size of promoters in bp and compute the proportion of hypo-/hyper-DMRs w.r.t to it

In [50]:
n_bases_total_promoter = (promoters.end - promoters.start).sum()

print(f'% Hypo-DMR w.r.t promoter = {(promoter_hypo_dmrs.end - promoter_hypo_dmrs.start).sum() / n_bases_total_promoter:.2%}')
print(f'% Hyper-DMR w.r.t promoter = {(promoter_hyper_dmrs.end - promoter_hyper_dmrs.start).sum() / n_bases_total_promoter:.2%}')

% Hypo-DMR w.r.t promoter = 3.87%
% Hyper-DMR w.r.t promoter = 0.11%


### Compute the total size of enhancers in bp and compute the proportion of hypo-/hyper-DMRs w.r.t to it

In [51]:
n_bases_total_enhancer = (enhancers.end - enhancers.start).sum()

print(f'% Hypo-DMR w.r.t enhancer = {(enhancer_hypo_dmrs.end - enhancer_hypo_dmrs.start).sum() / n_bases_total_enhancer:.2%}')
print(f'% Hyper-DMR w.r.t enhancer = {(enhancer_hyper_dmrs.end - enhancer_hyper_dmrs.start).sum() / n_bases_total_enhancer:.2%}')

% Hypo-DMR w.r.t enhancer = 0.28%
% Hyper-DMR w.r.t enhancer = 0.10%


### Other

In [93]:
!cat 'result/bed/enhancers.bed' 'result/bed/promoters.mm10.transcript.bed' |\
    bedtools sort -i stdin |\
    bedtools merge -i stdin > result/DMR-ADDITIONAL/promoter_enhancer.bed

In [94]:
pe = pd.read_csv('result/DMR-ADDITIONAL/promoter_enhancer.bed', sep='\t', names=['chrom', 'start', 'end', 'name'])

In [97]:
n_bases_total

2725521370

In [98]:
n_bases_total - (pe.end - pe.start).sum()

2345886037

In [105]:
!bedtools intersect -a 'result/dmr/ad_hypo.dmr.final.bed' -b 'result/bed/promoters.mm10.transcript.bed' 'result/bed/enhancers.bed' -v > 'result/DMR-ADDITIONAL/ad_hypo.dmr.other_region.bed'
!bedtools intersect -a 'result/dmr/ad_hyper.dmr.final.bed' -b 'result/bed/promoters.mm10.transcript.bed' 'result/bed/enhancers.bed' -v > 'result/DMR-ADDITIONAL/ad_hyper.dmr.other_region.bed'

In [108]:
hypo_other = pd.read_csv('result/DMR-ADDITIONAL/ad_hypo.dmr.other_region.bed', sep='\t', names=['chrom', 'start', 'end'])
hyper_other = pd.read_csv('result/DMR-ADDITIONAL/ad_hyper.dmr.other_region.bed', sep='\t', names=['chrom', 'start', 'end'])

In [111]:
(hypo_other.end - hypo_other.start).sum(), (hyper_other.end - hyper_other.start).sum()

(1888585, 738510)

In [110]:
hyper_other.shape

(696, 3)